<a href="https://colab.research.google.com/github/jhivandebenoit/Melanoma_Detection/blob/master/Melanoma_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install kaggle
!pip install tensorflow
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

2.8.0
1


In [45]:
!pip install livelossplot
!pip install keras_applications
!pip install opencv-python


In [46]:
!pip install -q kaggle

In [ ]:
#from google.colab import files 
#files.upload()


In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
#!kaggle datasets download -d nroman/melanoma-external-malignant-256


In [ ]:
! mkdir datasets

In [ ]:
! unzip melanoma-external-malignant-256.zip -d datasets

In [ ]:
import numpy as np
import pandas as pd
import cv2
from livelossplot import PlotLossesKeras
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K

In [ ]:
train_dir='/content/datasets/train/train/'
test_dir='/content/datasets/test/test/'
train_df=pd.read_csv('/content/datasets/train_concat.csv')
#test_df=pd.read_csv('/code/Melanoma detection/data/test.csv')

In [ ]:
train_df.head()


In [ ]:
train_df['anatom_site_general_challenge'].value_counts()

In [ ]:
train_df['target'].value_counts()

In [ ]:
#Taking about 2000~ images from the data set with a ratio of 3:1 benign to melanoma
benign_df = train_df[train_df['target']==0].sample(1500)
malignant_df = train_df[train_df['target']==1]

In [ ]:
train_df=pd.concat([benign_df,malignant_df])
train_df=train_df.reset_index() #stop old index being added as a column
train_df.head()


In [ ]:
#Creating dataframe of image addresses with their labels
labels=[]
train_data=[]
for i in range(train_df.shape[0]):
    train_data.append(train_dir + train_df['image_name'].iloc[i]+'.jpg')
    labels.append(train_df['target'].iloc[i])
df=pd.DataFrame(train_data)
df.columns=['images']
df['target']=labels
df.head()

In [ ]:
#Creating df of image addresses for testing, no labels
test_data=[]
for i in range(test_df.shape[0]):
    test_data.append(test_dir + test_df['image_name'].iloc[i]+'.jpg')
test_df=pd.DataFrame(test_data)
test_df.columns=['images']
test_df.head()

In [ ]:
#Splitting Labeled data for train/val
from  sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(df['images'],df['target'],test_size = 0.1, random_state = 1)
train_df=pd.DataFrame(x_train)
train_df.columns=['images']
train_df['target']=y_train

validation_df=pd.DataFrame(x_val)
validation_df.columns=['images']
validation_df['target']=y_val


In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_dataframe(
    train_df,
    x_col='images',
    y_col='target',
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')
val_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_dataframe(
    validation_df,
    x_col='images',
    y_col='target',
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')


In [ ]:
def vgg16_model( num_classes=None):

    model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x=Flatten()(model.output)
    output=Dense(1,activation='sigmoid')(x) # because we have to predict the AUC
    model=Model(model.input,output)
    
    return model

vgg_conv=vgg16_model(1)

In [ ]:
opt = Adam(learning_rate=1e-5)
vgg_conv.compile(loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()],optimizer=opt)
vgg_conv.summary()

In [ ]:
nb_epochs = 2
batch_size=8
nb_train_steps = train_df.shape[0]//batch_size
nb_val_steps=validation_df.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:

vgg_conv.fit(train_gen,
          epochs=nb_epochs,
          steps_per_epoch=nb_train_steps,
          validation_data=val_gen,
          callbacks=[PlotLossesKeras()],
          validation_steps=nb_val_steps,
          verbose=0)

In [ ]:
vgg_conv.save('model')

In [ ]:
!zip -r /content/file.zip /content/content
